In [2]:
import geopandas as gpd
import rasterio
import pandas as pd
from rasterio import features
import numpy as np
from shapely.geometry import Point
from affine import Affine
from skimage.graph import route_through_array
from pyproj import Transformer

In [4]:
# Step 1: Load the Shapefiles
districts = gpd.read_file('/Users/hanrui/Dropbox/Mac (3)/Documents/Infra&Nation/GIS/Ruggedness/india1881_1931_LinguisticMap.shp')
rail_1881 = gpd.read_file('/Users/hanrui/Dropbox/Mac (3)/Documents/Infra&Nation/GIS/Rail1881/Rail1881.shp')
treatment_rail = gpd.read_file('/Users/hanrui/Dropbox/Mac (3)/Documents/Infra&Nation/GIS/treatment_railway/Rail_1881_1931.shp')

if rail_1881.crs != treatment_rail.crs:
    treatment_rail = treatment_rail.to_crs(rail_1881.crs)

treatment_rail = treatment_rail[rail_1881.columns]
rail_1931 = pd.concat([rail_1881, treatment_rail], ignore_index=True)


In [6]:
# Step 2: Reproject All Layers to a Suitable Projected CRS
projected_crs = "EPSG:24379"  # India - onshore between 21°N and 28°N and west of 82°E.

# Reproject districts
if districts.crs != projected_crs:
    districts = districts.to_crs(projected_crs)

# Reproject railways
if rail_1881.crs != projected_crs:
    rail_1881 = rail_1881.to_crs(projected_crs)

if rail_1931.crs != projected_crs:
    rail_1931 = rail_1931.to_crs(projected_crs)


In [8]:
# Step 3: Define Latitude and Longitude

cities_data = [
    {'city_id': 1, 'city_name': 'Bombay', 'latitude': 19.0761, 'longitude': 72.8774},
    {'city_id': 2, 'city_name': 'Calcutta', 'latitude': 22.5726, 'longitude': 88.3639},
    {'city_id': 3, 'city_name': 'Madras', 'latitude': 13.0674, 'longitude': 80.2376},
    {'city_id': 4, 'city_name': 'Hyderabad', 'latitude': 17.3871, 'longitude': 78.4917},
    {'city_id': 5, 'city_name': 'Lucknow', 'latitude': 26.8500, 'longitude': 80.9499},
    {'city_id': 6, 'city_name': 'Benares', 'latitude': 25.3217, 'longitude': 82.9873},
    {'city_id': 7, 'city_name': 'Delhi', 'latitude': 28.6139, 'longitude': 77.2090},
    {'city_id': 8, 'city_name': 'Patna', 'latitude': 25.6127, 'longitude': 85.1589},
    {'city_id': 9, 'city_name': 'Agra', 'latitude': 27.1767, 'longitude': 78.0081},
    {'city_id': 10, 'city_name': 'Bangalore', 'latitude': 12.9716, 'longitude': 77.5946},
    {'city_id': 11, 'city_name': 'Amritsar', 'latitude': 31.6340, 'longitude': 74.8723},
    {'city_id': 12, 'city_name': 'Cawnpore', 'latitude': 26.4499, 'longitude': 80.3319},
    {'city_id': 13, 'city_name': 'Lahore', 'latitude': 31.5820, 'longitude': 74.3293},
    {'city_id': 14, 'city_name': 'Allahabad', 'latitude': 31.5820, 'longitude': 74.3293},
    {'city_id': 15, 'city_name': 'Jaipur', 'latitude': 26.9221, 'longitude': 75.7789},
    {'city_id': 16, 'city_name': 'Poona', 'latitude': 18.5167, 'longitude': 73.8563},
    {'city_id': 17, 'city_name': 'Ahmedabad', 'latitude': 23.0339, 'longitude': 72.5850},
    {'city_id': 18, 'city_name': 'Bareilly', 'latitude': 28.3757, 'longitude': 79.4360},
    {'city_id': 19, 'city_name': 'Surat', 'latitude': 21.1702, 'longitude': 72.8311},
    {'city_id': 20, 'city_name': 'Howrah', 'latitude': 22.5958, 'longitude': 88.2636}
]


In [10]:
cities_data = pd.DataFrame(cities_data)
# Create a GeoDataFrame for the cities
geometry = [Point(lon, lat) for lon, lat in zip(cities_data['longitude'], cities_data['latitude'])]
cities_gdf = gpd.GeoDataFrame(cities_data, geometry=geometry, crs="EPSG:4326")

# Reproject cities to the projected CRS
cities_gdf = cities_gdf.to_crs(projected_crs)


In [12]:
# Step 4: Calculate District Centroids
districts['centroid'] = districts.centroid
centroids = districts.copy()
centroids.set_geometry('centroid', inplace=True)


In [14]:
# Step 5: Define the Raster Grid Parameters
# Define the pixel size (in meters)
pixel_size = 1000  # Adjust as needed

# Calculate the bounds of the raster (minx, miny, maxx, maxy)
districts_bounds = districts.total_bounds
cities_bounds = cities_gdf.total_bounds
minx = min(districts_bounds[0], cities_bounds[0])
miny = min(districts_bounds[1], cities_bounds[1])
maxx = max(districts_bounds[2], cities_bounds[2])
maxy = max(districts_bounds[3], cities_bounds[3])

# Calculate the number of rows and columns in the raster
width = int((maxx - minx) / pixel_size) + 1
height = int((maxy - miny) / pixel_size) + 1

# Define the affine transformation (maps pixel coordinates to world coordinates)
transform = Affine.translation(minx, miny) * Affine.scale(pixel_size, pixel_size)


In [16]:
# Step 6: Create Cost Rasters for 1881 and 1931
# Initialize cost rasters with default cost 2.5 (non-railway areas)
# Based on the estimation from Donalson (2018), cost of railway is normalized to 1, cost on roads is around 2.5 
cost_raster_1881 = np.full((height, width), 2.5, dtype=np.float32)
cost_raster_1931 = np.full((height, width), 2.5, dtype=np.float32)

# Function to rasterize railway lines onto the cost raster
def rasterize_railways(rail_gdf, cost_raster):
    shapes = ((geom, 1) for geom in rail_gdf.geometry)
    rasterized = features.rasterize(
        shapes=shapes,
        out_shape=cost_raster.shape,
        transform=transform,
        fill=2.5,
        all_touched=True,
        dtype=np.float32
    )
    return rasterized

# Rasterize railway lines for 1881 and 1931
cost_raster_1881 = rasterize_railways(rail_1881, cost_raster_1881)
cost_raster_1931 = rasterize_railways(rail_1931, cost_raster_1931)


In [18]:
# Step 7: Find Raster Indices for Cities and District Centroids
def world_to_pixel(coords, transform):
    x, y = coords
    col, row = ~transform * (x, y)
    return int(row), int(col)

# Get cities' raster indices
city_rows = []
city_cols = []
for geom in cities_gdf.geometry:
    row, col = world_to_pixel((geom.x, geom.y), transform)
    city_rows.append(row)
    city_cols.append(col)

# Get centroids' raster indices
centroid_rows = []
centroid_cols = []
for geom in districts['centroid']:
    row, col = world_to_pixel((geom.x, geom.y), transform)
    centroid_rows.append(row)
    centroid_cols.append(col)

In [20]:
# Step 8: Perform Least Cost Path Analysis Using Dijkstra's Algorithm
results = []

# For each year
for year, cost_raster in zip([1881, 1931], [cost_raster_1881, cost_raster_1931]):
   # print(f"Processing year {year}")
    # For each district centroid
    for district_idx, (district_id, district_name, cent_row, cent_col) in enumerate(
        zip(districts.index, districts['Name'], centroid_rows, centroid_cols)
    ):
        # For each city
        for city_idx, (city_id, city_name, city_row, city_col) in enumerate(
            zip(cities_gdf['city_id'], cities_gdf['city_name'], city_rows, city_cols)
        ):
           # print(f"Processing district {district_name} to city {city_name} for year {year}")
            try:
                indices, weight = route_through_array(
                    cost_raster,
                    (cent_row, cent_col),
                    (city_row, city_col),
                    fully_connected=True
                )
                least_cost = weight
            except Exception as e:
                print(f"Error processing district {district_name} to city {city_name} for {year}: {e}")
                least_cost = np.nan
            # Append the result
            results.append({
                'district_id': district_id,
                'district_name': district_name,
                'city_id': city_id,
                'city_name': city_name,
                'year': year,
                'least_cost': least_cost
            })


In [22]:
trade_cost = pd.DataFrame(results)

In [24]:
trade_cost

,district_id,district_name,city_id,city_name,year,least_cost
0,0,Censused Area Balipara Frontier Tract,1,Bombay,1881,3471.635008
1,0,Censused Area Balipara Frontier Tract,2,Calcutta,1881,1485.095507
2,0,Censused Area Balipara Frontier Tract,3,Madras,1881,4524.715094
3,0,Censused Area Balipara Frontier Tract,4,Hyderabad,1881,4003.104615
4,0,Censused Area Balipara Frontier Tract,5,Lucknow,1881,2307.857177
...,...,...,...,...,...,...
17195,429,Baroda City and Cantonment,16,Poona,1931,542.805393
17196,429,Baroda City and Cantonment,17,Ahmedabad,1931,107.741901
17197,429,Baroda City and Cantonment,18,Bareilly,1931,1096.427326
17198,429,Baroda City and Cantonment,19,Surat,1931,146.543416


In [26]:
trade_cost_wide = trade_cost.pivot(index=['district_id', 'district_name', 'year'], columns='city_name', values='least_cost')
trade_cost_wide.columns = [f"{col}" for col in trade_cost_wide.columns]
trade_cost_wide = trade_cost_wide.reset_index()

In [28]:
districts = pd.merge(districts, trade_cost_wide, left_on=['Name'], right_on=['district_name'], how='left')

In [30]:
districts

,Name,DistCode,NEAR_DIST,Shape_Leng,Shape_Area,FID_,DIST,path_FID,path_DIST,NEAR_FID,...,Delhi,Howrah,Hyderabad,Jaipur,Lahore,Lucknow,Madras,Patna,Poona,Surat
0,Censused Area Balipara Frontier Tract,30108,136228.326464,3.944586,0.184502,20,387533.734533,20,387533.734533,1,...,2787.229327,1506.034847,4003.104615,2842.712823,3425.108096,2307.857177,4524.715094,1718.156600,3547.404038,3671.645199
1,Censused Area Balipara Frontier Tract,30108,136228.326464,3.944586,0.184502,20,387533.734533,20,387533.734533,1,...,2112.065240,1042.217388,2647.266340,2177.959901,2552.140186,1594.370237,2831.851387,1067.646320,2894.065330,2737.659215
2,Censused Area Sadiya Frontier Tract,30107,386873.985441,4.916460,0.421089,20,635225.192956,20,635225.192956,1,...,3523.417277,2137.108619,4739.292565,3578.900773,4161.296047,3044.045128,5260.903045,2454.344551,4283.591988,4407.833150
3,Censused Area Sadiya Frontier Tract,30107,386873.985441,4.916460,0.421089,20,635225.192956,20,635225.192956,1,...,2465.954634,1339.692568,2957.698410,2531.849295,2906.029580,1948.259631,3129.326567,1421.535713,3247.954724,3091.548609
4,Uncensused Area Balipara Frontier Tract,0,146645.615299,9.249353,2.732980,20,397813.271861,20,397813.271861,1,...,2893.178641,1721.819118,4109.053929,2948.662137,3531.057411,2413.806491,4630.664408,1824.105915,3653.353352,3777.594514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,Orissa States,70601,145671.520364,35.696529,6.241826,19,131543.576425,19,131543.576425,3,...,1655.347420,653.851857,1406.364140,1623.375594,2163.324908,1285.073773,1623.360876,903.266594,1814.480410,1658.074296
856,Trans-frontier Posts,150106,334699.427499,26.957917,6.134666,17,315315.901286,17,315315.901286,4,...,1370.610876,2954.227325,3550.682736,1680.527921,673.083874,1844.857177,4072.293215,2378.641972,2971.266430,2575.004453
857,Trans-frontier Posts,150106,334699.427499,26.957917,6.134666,17,315315.901286,17,315315.901286,4,...,1097.067676,2550.358746,2803.992783,1301.158819,581.451100,1542.445922,3424.647418,2089.421625,2511.121679,2114.859703
858,Baroda City and Cantonment,190101,67260.127879,0.296932,0.006642,12,0.000000,12,0.000000,8,...,1081.092802,2453.267080,1156.790241,750.175757,1761.962950,1357.093848,1678.400720,1877.681727,542.805393,146.543416
